## SARiSC Laboratorium

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from typing import List
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold
from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.model_selection import StratifiedKFold


    RRI (RR interval): Jest to odstęp czasu między kolejnymi skurczami serca, mierzony w milisekundach (ms). RRI jest miarą zmienności rytmu serca, która może być przydatna do oceny regulacji autonomicznej.

    RR-systolic: Odnosi się do ciśnienia skurczowego krwi, czyli najwyższego ciśnienia krwi w naczyniach krwionośnych, wyrażonego w milimetrach słupa rtęci (mmHg), które towarzyszy skurczowi serca.

    RR-diastolic: Odnosi się do ciśnienia rozkurczowego krwi, czyli najniższego ciśnienia krwi w naczyniach krwionośnych, wyrażonego w milimetrach słupa rtęci (mmHg), które występuje podczas rozkurczu serca.

    RR-mean: Odnosi się do ciśnienia średniego krwi, czyli średniego ciśnienia krwi w naczyniach krwionośnych w okresie cyklu sercowego, wyrażonego w milimetrach słupa rtęci (mmHg).

    IBI (Interbeat Interval): Jest to również odstęp czasu między kolejnymi skurczami serca, mierzony w milisekundach (ms). IBI jest również miarą zmienności rytmu serca i jest równoważne RRI.

In [2]:
class ColNames:
    file_name = 'file name'
    sex = 'SEX [nominal codes: "1" woman; "2" man]'
    file = 'file'

class Paths:
    HRAecplorer = 'HRAexplorer/'
    Runs = HRAecplorer + 'Runs.xlsx'
    Spectral =  HRAecplorer + 'Spectral.xlsx'
    PoincrePlot = HRAecplorer + 'PoincrePlot.xlsx'
    HYPOL_RECORDINGS = 'HYPOL RECORDINGS/'
    HYPOL_Main = 'HYPOL clinical characteristics.xls'


class MyData:
    def __init__(self, target, value):
        self.target = target
        self.value = value

    def DFToVector(self):
        if isinstance(self.value, pd.DataFrame):
            return self.value.to_numpy().ravel()

    def DFModify(self):
        if isinstance(self.value, pd.DataFrame):
            columns_to_remove = [ColNames.time_ms, ColNames.rr_flags]
            self.value = self.value.drop(columns=columns_to_remove)


In [22]:
def crossValid(values, targets, splits: int, kernel: str, adasyn_multiply: int=5):
    skf = StratifiedKFold(n_splits=splits, random_state=42, shuffle=True)

    # Przeprowadź walidację krzyżową
    train_accuracies = []
    test_accuracies = []

    for train_index, test_index in skf.split(values, targets):
        # Podział danych na zbiory treningowy i testowy
        X_train, X_test = values[train_index], values[test_index]
        y_train, y_test = targets[train_index], targets[test_index]
        
        # Obliczanie proporcji oversamplingu dla każdej klasy
        unique_classes, class_counts = np.unique(y_train, return_counts=True)
        count = np.max(class_counts) * adasyn_multiply
        target_ratio = {cls: count for cls in unique_classes}
        print(f'target_ratio: {target_ratio}')
        # Zastosowanie ADASYN do zwiększenia liczby danych tylko dla zbioru treningowego
        adasyn = ADASYN(sampling_strategy=target_ratio, random_state=42)
        X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)
        
        # Trenowanie modelu klasyfikacji, np. klasyfikatora SVM
        clf = svm.SVC(kernel=kernel)
        clf.fit(X_train_resampled, y_train_resampled)
        
        # Ocena dokładności na danych treningowych
        train_accuracy = clf.score(X_train_resampled, y_train_resampled)
        train_accuracies.append(train_accuracy)
        
        # Ocena dokładności na danych testowych
        test_accuracy = clf.score(X_test, y_test)
        test_accuracies.append(test_accuracy)
        print("Dokładność modelu SVM na danych treningowych: {:.2f}%".format(train_accuracy * 100))
        print("Dokładność modelu SVM na danych testowych: {:.2f}%".format(test_accuracy * 100))

    # Obliczenie średniej dokładności dla danych treningowych i testowych
    avg_train_accuracy = np.mean(train_accuracies)
    avg_test_accuracy = np.mean(test_accuracies)

    print("Średnia dokładność modelu SVM na danych treningowych: {:.2f}%".format(avg_train_accuracy * 100))
    print("Średnia dokładność modelu SVM na danych testowych: {:.2f}%".format(avg_test_accuracy * 100))


def crossValidKNN(values, targets, splits: int, nn: int, adasyn_multiply: int=5):
    skf = StratifiedKFold(n_splits=splits, random_state=42, shuffle=True)

    # Przeprowadź walidację krzyżową
    train_accuracies = []
    test_accuracies = []

    for train_index, test_index in skf.split(values, targets):
        # Podział danych na zbiory treningowy i testowy
        X_train, X_test = values[train_index], values[test_index]
        y_train, y_test = targets[train_index], targets[test_index]
        
        # Obliczanie proporcji oversamplingu dla każdej klasy
        unique_classes, class_counts = np.unique(y_train, return_counts=True)
        count = np.max(class_counts) * adasyn_multiply
        target_ratio = {cls: count for cls in unique_classes}
        print(f'target_ratio: {target_ratio}')
        # Zastosowanie ADASYN do zwiększenia liczby danych tylko dla zbioru treningowego
        adasyn = ADASYN(sampling_strategy=target_ratio, random_state=42)
        X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)
        
        # Trenowanie modelu klasyfikacji, np. klasyfikatora SVM
        knn = KNeighborsClassifier(nn)
        knn.fit(X_train_resampled, y_train_resampled)
        
        # Ocena dokładności na danych treningowych
        train_accuracy = knn.score(X_train_resampled, y_train_resampled)
        train_accuracies.append(train_accuracy)
        
        # Ocena dokładności na danych testowych
        test_accuracy = knn.score(X_test, y_test)
        test_accuracies.append(test_accuracy)
        print("Dokładność modelu knn na danych treningowych: {:.2f}%".format(train_accuracy * 100))
        print("Dokładność modelu knn na danych testowych: {:.2f}%".format(test_accuracy * 100))

    # Obliczenie średniej dokładności dla danych treningowych i testowych
    avg_train_accuracy = np.mean(train_accuracies)
    avg_test_accuracy = np.mean(test_accuracies)

    print("Średnia dokładność modelu knn na danych treningowych: {:.2f}%".format(avg_train_accuracy * 100))
    print("Średnia dokładność modelu knn na danych testowych: {:.2f}%".format(avg_test_accuracy * 100))


def CoefKFold(values, targets):
    clf = svm.SVC(kernel='linear')

    # Użyjemy KFold do ręcznej implementacji walidacji krzyżowej, aby mieć dostęp do modeli
    kf = KFold(n_splits=5)
    scores = []
    coef_list = []

    for i, (train_index, test_index) in enumerate(kf.split(values)):
        X_train, X_test = values[train_index], values[test_index]
        y_train, y_test = targets[train_index], targets[test_index]
        
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        scores.append(score)
        
        coef_list.append(clf.coef_[0])
        
        print(f"Wynik dla foldu {i+1}:", score)
        print(f"Ważność cech dla foldu {i+1}: {clf.coef_[0]}")
        
    print("Średnia dokładność walidacji krzyżowej:", np.mean(scores))

    # Jeśli chcesz zobaczyć średnią ważność cech:
    mean_coef = np.mean(coef_list, axis=0)
    print("Średnia ważność cech:", mean_coef)
    return mean_coef

In [4]:
def get_all_HRA_explorer_df():
    df_runs = pd.read_excel(Paths.Runs)
    df_poincre = pd.read_excel(Paths.PoincrePlot)
    df_spectral = pd.read_excel(Paths.Spectral)
    df = df_runs.merge(df_poincre, on=ColNames.file).merge(df_spectral, on=ColNames.file)
    return df
def get_df_main():
    df_main = pd.read_excel(Paths.HYPOL_Main)
    df_main.rename(columns={ColNames.file_name: ColNames.file}, inplace=True)
    return df_main


In [5]:
df_main = get_df_main()

print(df_main.head())

        file  SEX [nominal codes: "1" woman; "2" man]  Age [years]   
0  ag016.rea                                        1           21  \
1  ag019.rea                                        1           26   
2  ag023.rea                                        1           24   
3  ag024.rea                                        2           24   
4  ag025.rea                                        2           24   

   BMI [kg/m2]  body Height [cm]  body WEIGHT [kg]  Pulse Rate [bpm]   
0        21.80               170                63                65  \
1        21.01               169                60                61   
2        20.32               166                56                66   
3        24.74               197                96                54   
4        22.49               170                65                55   

   Brachial SBP [mmHg]  Brachial DBP [mmHg]  Brachial MBP [mmHg]   
0                  115                   71                   84  \
1         

In [6]:
df_value = get_all_HRA_explorer_df()


In [7]:

df : pd.DataFrame = df_main[[ColNames.file, ColNames.sex]]
df = df.merge(df_value, on=ColNames.file)


In [12]:
df_values : pd.DataFrame = df.drop(columns=[ColNames.file, ColNames.sex])
np_values = df_values.to_numpy()
np_targets = df[[ColNames.sex]].to_numpy().ravel()


In [13]:
print(df.info())
print(np_values.shape)
print(np_targets.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 52 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   file                                     276 non-null    object 
 1   SEX [nominal codes: "1" woman; "2" man]  276 non-null    int64  
 2   DR1                                      276 non-null    int64  
 3   DR2                                      276 non-null    int64  
 4   DR3                                      276 non-null    int64  
 5   DR4                                      276 non-null    int64  
 6   DR5                                      276 non-null    int64  
 7   DR6                                      276 non-null    int64  
 8   DR7                                      276 non-null    int64  
 9   DR8                                      276 non-null    int64  
 10  DR9                                      276 non-n

In [14]:
print(df_values.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 50 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DR1      276 non-null    int64  
 1   DR2      276 non-null    int64  
 2   DR3      276 non-null    int64  
 3   DR4      276 non-null    int64  
 4   DR5      276 non-null    int64  
 5   DR6      276 non-null    int64  
 6   DR7      276 non-null    int64  
 7   DR8      276 non-null    int64  
 8   DR9      276 non-null    int64  
 9   DR10     276 non-null    int64  
 10  DR11     276 non-null    int64  
 11  DR12     276 non-null    int64  
 12  AR1      276 non-null    int64  
 13  AR2      276 non-null    int64  
 14  AR3      276 non-null    int64  
 15  AR4      276 non-null    int64  
 16  AR5      276 non-null    int64  
 17  AR6      276 non-null    int64  
 18  AR7      276 non-null    int64  
 19  AR8      276 non-null    int64  
 20  AR9      276 non-null    int64  
 21  AR10     276 non

In [23]:
coef = CoefKFold(np_values, np_targets)

Wynik dla foldu 1: 0.6071428571428571
Ważność cech dla foldu 1: [ 3.16401143e-01  2.08324272e-01  2.39910287e-01  1.26231426e-01
 -4.56516907e-01  1.36552651e+00  1.22251121e+00 -3.29690749e-01
  5.85397206e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.24991542e-01 -1.81915818e-01 -1.95077694e-01 -1.95761346e-01
 -1.22159964e-01  4.55709477e-01 -1.52437137e-01 -1.63813022e+00
  3.52509838e+00  1.13481799e+00 -8.72432221e-01  1.65271703e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.09207373e-01
  4.66734507e-01 -1.16150593e-01 -5.33721785e-01  3.35620901e+00
  1.75288357e+00  2.18422434e-01 -7.67885221e-01  1.66590686e-02
 -7.73960086e-01  1.35205159e-01 -1.81610101e-01  5.29192352e-01
 -7.00075693e-01  1.90133740e-01 -5.16163618e-01  4.87374183e-01
 -1.12164588e-01 -2.05928868e-03 -8.41938737e-03  3.71395570e-03
  9.36665466e-03  2.60193442e-03]
Wynik dla foldu 2: 0.7090909090909091
Ważność cech dla foldu 2: [ 2.05819458e-01  1.72780981e-01  1.95106834e-01  1.067403

In [24]:
for i, c in enumerate(coef):
    print(f"{i}: {df_values.columns[i]} - {c}")

0: DR1 - 0.24682058835574594
1: DR2 - 0.19742746913881887
2: DR3 - 0.21819121234775452
3: DR4 - 0.10683680729470132
4: DR5 - -0.5824532014931909
5: DR6 - 1.0494919932014053
6: DR7 - -0.23329141732661274
7: DR8 - 0.6945606484452288
8: DR9 - 0.2777617898370795
9: DR10 - 0.0
10: DR11 - 0.0
11: DR12 - 0.0
12: AR1 - -0.20426794144438531
13: AR2 - -0.1882011483407382
14: AR3 - -0.16168837444600684
15: AR4 - -0.07627989921513834
16: AR5 - 0.02150182432364973
17: AR6 - -0.036711040664419235
18: AR7 - -0.03612672839901969
19: AR8 - -1.1739142131501448
20: AR9 - 0.4486657015602596
21: AR10 - -0.5780642106901961
22: AR11 - -0.30837366971167574
23: AR12 - 0.5110281757105316
24: AR13 - 0.0
25: AR14 - 0.13447321386026903
26: AR15 - 0.39122128538164624
27: N1 - -0.01072928099683601
28: N2 - 0.3470762372770707
29: N3 - -0.3175778141596008
30: DR_MAX - -1.0141713782392596
31: AR_MAX - 2.7425711187782342
32: N_MAX - -0.0804770893595356
33: SDNN - 1.1064240497262374
34: SD1 - -0.762499632031151
35: SD2 -

In [26]:
column_to_drop = df_values.columns[[9,10, 11, 24, 45]]
print(column_to_drop)
df_values_droped : pd.DataFrame = df_values.drop(column_to_drop, axis=1)
print(df_values_droped.info())

Index(['DR10', 'DR11', 'DR12', 'AR13', 'ULF'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 45 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DR1      276 non-null    int64  
 1   DR2      276 non-null    int64  
 2   DR3      276 non-null    int64  
 3   DR4      276 non-null    int64  
 4   DR5      276 non-null    int64  
 5   DR6      276 non-null    int64  
 6   DR7      276 non-null    int64  
 7   DR8      276 non-null    int64  
 8   DR9      276 non-null    int64  
 9   AR1      276 non-null    int64  
 10  AR2      276 non-null    int64  
 11  AR3      276 non-null    int64  
 12  AR4      276 non-null    int64  
 13  AR5      276 non-null    int64  
 14  AR6      276 non-null    int64  
 15  AR7      276 non-null    int64  
 16  AR8      276 non-null    int64  
 17  AR9      276 non-null    int64  
 18  AR10     276 non-null    int64  
 19  AR11     276 non-null    int6

In [27]:
CoefKFold(df_values_droped.to_numpy(), np_targets)

Wynik dla foldu 1: 0.5
Ważność cech dla foldu 1: [ 4.83558753e-01  3.53997819e-01  2.07104084e-01  5.42021747e-01
 -1.12170065e+00  2.52664551e+00 -2.68774930e-01 -2.93372237e-01
 -2.21232020e+00 -3.84441635e-01 -3.70907730e-01 -3.37114339e-01
 -6.76647958e-01 -4.69433656e-01  1.00971723e+00 -7.66032280e-01
 -1.66999284e+00 -1.39185113e+00 -8.63515124e-01 -1.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.87765400e-01
 -1.83562714e+00 -1.17383306e-01 -9.75677378e-01  3.04913291e+00
 -1.71635599e+00 -1.97269112e+00 -2.64806149e-01  1.87485404e+00
 -2.73997117e-01  8.17102625e-02 -2.75337269e+00 -3.87985015e-01
 -1.06510737e+00  1.41012311e+00  1.13889782e+00  1.08663427e+00
 -2.33904316e-02 -1.30441825e-02  9.56856522e-03  9.49383188e-03
 -1.32901980e-03]
Wynik dla foldu 2: 0.6909090909090909
Ważność cech dla foldu 2: [-5.67472388e-01 -5.90787525e-01 -5.69535188e-01 -6.52180984e-01
 -5.84226086e-01 -5.11394868e-01 -2.31054277e+00  1.10485623e+00
  2.00000000e+00  5.7654

array([-3.47875876e-04, -6.50340491e-02, -1.01003145e-01, -1.71998936e-02,
       -8.04538236e-01,  9.09057597e-01, -1.26702855e+00,  5.10538207e-01,
       -6.52671036e-01,  4.53653044e-02,  7.48190659e-02,  1.46434171e-01,
        1.44570279e-01,  5.80305845e-02,  4.36862334e-01, -8.49293173e-02,
       -5.72078167e-01, -2.59380920e-02, -1.38178883e+00, -8.00000000e-01,
       -3.24947866e-01,  2.45244169e-02,  3.07962935e-01,  1.35389284e-01,
       -4.82288700e-01, -4.40928804e-03, -8.17609971e-01,  2.80797912e+00,
       -1.15607172e+00,  1.48506569e-03, -4.08495381e-01,  2.66948824e-01,
       -4.13230033e-01,  1.10425832e-01, -5.30005750e-01,  3.83249878e-01,
       -4.90665710e-02,  2.93043870e-01, -1.00199392e-01,  2.09623621e-01,
       -2.27990048e-02, -9.29379838e-03,  2.12603018e-03, -6.56641446e-03,
        2.78843092e-03])

In [28]:
crossValid(df_values_droped.to_numpy(), np_targets, 10, 'linear', adasyn_multiply=5)

target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 71.30%
Dokładność modelu SVM na danych testowych: 53.57%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 73.14%
Dokładność modelu SVM na danych testowych: 75.00%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 74.36%
Dokładność modelu SVM na danych testowych: 42.86%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 74.49%
Dokładność modelu SVM na danych testowych: 67.86%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 74.48%
Dokładność modelu SVM na danych testowych: 75.00%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 76.69%
Dokładność modelu SVM na danych testowych: 64.29%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 76.64%
Dokładność modelu SVM na danych testowych: 51.85%
target_ratio: {1: 665, 2: 665}
Dokładność modelu SVM na danych

In [29]:
crossValid(df_values_droped.to_numpy(), np_targets, 10, 'rbf', adasyn_multiply=5)

target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 58.59%
Dokładność modelu SVM na danych testowych: 57.14%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 58.90%
Dokładność modelu SVM na danych testowych: 53.57%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 60.98%
Dokładność modelu SVM na danych testowych: 64.29%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 59.22%
Dokładność modelu SVM na danych testowych: 46.43%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 63.62%
Dokładność modelu SVM na danych testowych: 67.86%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 57.68%
Dokładność modelu SVM na danych testowych: 60.71%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 58.73%
Dokładność modelu SVM na danych testowych: 48.15%
target_ratio: {1: 665, 2: 665}
Dokładność modelu SVM na danych

In [31]:
column_to_drop_2 = df_values.columns[[9,10, 11, 15,16,17,18, 24, 27, 44, 45,46,47,48,49]]
print(column_to_drop_2)
df_values_droped_2 : pd.DataFrame = df_values.drop(column_to_drop_2, axis=1)
print(df_values_droped_2.info())
np_values_droped_2 = df_values_droped_2.to_numpy()

Index(['DR10', 'DR11', 'DR12', 'AR4', 'AR5', 'AR6', 'AR7', 'AR13', 'N1', 'PI',
       'ULF', 'VLF', 'LF', 'HF', 'TP'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DR1      276 non-null    int64  
 1   DR2      276 non-null    int64  
 2   DR3      276 non-null    int64  
 3   DR4      276 non-null    int64  
 4   DR5      276 non-null    int64  
 5   DR6      276 non-null    int64  
 6   DR7      276 non-null    int64  
 7   DR8      276 non-null    int64  
 8   DR9      276 non-null    int64  
 9   AR1      276 non-null    int64  
 10  AR2      276 non-null    int64  
 11  AR3      276 non-null    int64  
 12  AR8      276 non-null    int64  
 13  AR9      276 non-null    int64  
 14  AR10     276 non-null    int64  
 15  AR11     276 non-null    int64  
 16  AR12     276 non-null    int64  
 17  AR14     276 non-null    in

In [35]:
coef = CoefKFold(np_values_droped_2, np_targets)

Wynik dla foldu 1: 0.6785714285714286
Ważność cech dla foldu 1: [ 0.01426007  0.00865353  0.00561863  0.00192909 -0.012671    0.18723978
 -0.13483075 -0.15975575  0.0993437  -0.01004322 -0.00825473 -0.006245
  0.06392608 -0.0892934   0.19142005 -0.55059597  0.          0.
  0.          0.07057903 -0.16412193 -0.09312215  0.22546329  0.09800647
 -0.17064657 -0.3417302   0.12065286 -0.34898463  0.01125182  0.06773105
 -0.33489278  0.60589771  0.40432977 -0.05840274  0.23716772]
Wynik dla foldu 2: 0.7636363636363637
Ważność cech dla foldu 2: [ 0.00488945  0.00228305  0.00216599  0.00111194 -0.04959768  0.13375448
 -0.1809834  -0.1377737   0.36599707 -0.00125076 -0.00375638  0.00470195
  0.01984397 -0.0991677  -0.39345465 -0.88873257 -0.21461763  0.
  0.          0.0422415  -0.21767531 -0.16002461  0.44022848  0.18295366
 -0.02180361 -0.37713486 -0.21727627 -0.38103365  0.00993968  0.1533926
  0.10923245  0.49479943  0.38025336 -0.00391617  0.18600951]
Wynik dla foldu 3: 0.6545454545454545

In [36]:
for i, c in enumerate(coef):
    print(f"{i}: {df_values_droped_2.columns[i]} - {c}")

0: DR1 - 0.013426008387924071
1: DR2 - 0.009793517725586297
2: DR3 - 0.008261680725122744
3: DR4 - 0.0035931711541795862
4: DR5 - -0.03852434276279446
5: DR6 - 0.1309290761924981
6: DR7 - -0.13669540796992266
7: DR8 - -0.02162380688717085
8: DR9 - 0.15777327220689963
9: AR1 - -0.010150967079479756
10: AR2 - -0.011600173535030933
11: AR3 - -0.0049102124342788045
12: AR8 - 0.029412618910179322
13: AR9 - -0.11026679530299718
14: AR10 - -0.1527055125524296
15: AR11 - -0.4211438223183296
16: AR12 - -0.0269876482418956
17: AR14 - 0.22948388269309375
18: AR15 - 0.036870608990331476
19: N2 - 0.030116639853093475
20: N3 - -0.07607041367331159
21: DR_MAX - -0.16423531508776462
22: AR_MAX - 0.3031558651912654
23: N_MAX - 0.16471665875211086
24: SDNN - -0.049392328477611616
25: SD1 - -0.30657296976840803
26: SD2 - -0.13563081585135847
27: SD1I - -0.31063372279331036
28: MEAN_RR - 0.01072337735504334
29: SDNNd - 0.0805557660073319
30: SDNNa - 5.994946997986972e-05
31: SD1d - 0.48741693263362224
32:

In [33]:
crossValid(np_values_droped_2, np_targets, 10, 'linear', adasyn_multiply=5)

target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 75.08%
Dokładność modelu SVM na danych testowych: 60.71%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 70.68%
Dokładność modelu SVM na danych testowych: 71.43%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 75.63%
Dokładność modelu SVM na danych testowych: 60.71%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 69.82%
Dokładność modelu SVM na danych testowych: 57.14%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 72.00%
Dokładność modelu SVM na danych testowych: 67.86%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 72.27%
Dokładność modelu SVM na danych testowych: 67.86%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 74.46%
Dokładność modelu SVM na danych testowych: 48.15%
target_ratio: {1: 665, 2: 665}
Dokładność modelu SVM na danych

In [34]:
crossValid(np_values_droped_2, np_targets, 10, 'rbf', adasyn_multiply=5)

target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 58.33%
Dokładność modelu SVM na danych testowych: 64.29%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 58.26%
Dokładność modelu SVM na danych testowych: 64.29%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 61.62%
Dokładność modelu SVM na danych testowych: 53.57%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 59.69%
Dokładność modelu SVM na danych testowych: 75.00%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 59.94%
Dokładność modelu SVM na danych testowych: 71.43%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 58.63%
Dokładność modelu SVM na danych testowych: 67.86%
target_ratio: {1: 660, 2: 660}
Dokładność modelu SVM na danych treningowych: 60.34%
Dokładność modelu SVM na danych testowych: 51.85%
target_ratio: {1: 665, 2: 665}
Dokładność modelu SVM na danych

In [38]:
column_to_drop_3 = df_values_droped_2.columns[[1,2,3,11,30]]
print(column_to_drop_3)
df_values_droped_3 : pd.DataFrame = df_values_droped_2.drop(column_to_drop_3, axis=1)
print(df_values_droped_3.info())
np_values_droped_3 = df_values_droped_3.to_numpy()

Index(['DR2', 'DR3', 'DR4', 'AR3', 'SDNNa'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 30 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DR1      276 non-null    int64  
 1   DR5      276 non-null    int64  
 2   DR6      276 non-null    int64  
 3   DR7      276 non-null    int64  
 4   DR8      276 non-null    int64  
 5   DR9      276 non-null    int64  
 6   AR1      276 non-null    int64  
 7   AR2      276 non-null    int64  
 8   AR8      276 non-null    int64  
 9   AR9      276 non-null    int64  
 10  AR10     276 non-null    int64  
 11  AR11     276 non-null    int64  
 12  AR12     276 non-null    int64  
 13  AR14     276 non-null    int64  
 14  AR15     276 non-null    int64  
 15  N2       276 non-null    int64  
 16  N3       276 non-null    int64  
 17  DR_MAX   276 non-null    int64  
 18  AR_MAX   276 non-null    int64  
 19  N_MAX    276 non-null    int64 

In [39]:
coef = CoefKFold(np_values_droped_3, np_targets)

Wynik dla foldu 1: 0.6785714285714286
Ważność cech dla foldu 1: [ 0.00481614 -0.02020053  0.16932454  0.00139212 -0.18291027 -0.31967986
  0.00192241 -0.00300719 -0.03719459  0.19593635  0.29463082 -0.79885416
  0.          0.01987964  0.         -0.01587104 -0.07269252 -0.03702244
  0.1915076  -0.00820762 -0.63719891 -0.31544855  0.31066353 -0.3186523
  0.01156432  0.20668052  0.65221532  0.33831392 -0.09649404  0.11571624]
Wynik dla foldu 2: 0.7818181818181819
Ważność cech dla foldu 2: [ 1.27889218e-03 -6.97617082e-02  1.75477008e-01 -1.86270188e-02
 -1.79318334e-02  6.13477376e-01  4.09123382e-04 -9.88784084e-04
  6.78993260e-02 -1.45950687e-01 -6.20924412e-01 -8.33237765e-01
 -9.70484016e-01  0.00000000e+00  0.00000000e+00  1.65645215e-02
 -4.91691036e-02 -2.04940328e-01  4.68057536e-01 -1.11069823e-01
 -2.86138338e-01 -2.95784619e-01 -2.51778812e-01 -3.00950415e-01
  8.91231514e-03  4.82871240e-01  3.16523322e-01  3.81418284e-01
 -8.22745630e-02  4.07090126e-01]
Wynik dla foldu 3:

In [45]:
crossValid(np_values_droped_2, np_targets, 10, 'linear', adasyn_multiply=15)

target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 76.21%
Dokładność modelu SVM na danych testowych: 60.71%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 69.21%
Dokładność modelu SVM na danych testowych: 82.14%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 75.08%
Dokładność modelu SVM na danych testowych: 57.14%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 72.12%
Dokładność modelu SVM na danych testowych: 60.71%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 73.28%
Dokładność modelu SVM na danych testowych: 78.57%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 72.23%
Dokładność modelu SVM na danych testowych: 60.71%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 74.41%
Dokładność modelu SVM na danych testowych: 44.44%
target_ratio: {1: 1995, 2: 1995}
Dokładność mode

In [44]:
crossValid(np_values_droped_2, np_targets, 10, 'rbf', adasyn_multiply=15)

target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 65.63%
Dokładność modelu SVM na danych testowych: 64.29%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 62.08%
Dokładność modelu SVM na danych testowych: 75.00%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 66.57%
Dokładność modelu SVM na danych testowych: 50.00%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 65.91%
Dokładność modelu SVM na danych testowych: 78.57%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 64.57%
Dokładność modelu SVM na danych testowych: 78.57%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 63.14%
Dokładność modelu SVM na danych testowych: 71.43%
target_ratio: {1: 1980, 2: 1980}
Dokładność modelu SVM na danych treningowych: 67.18%
Dokładność modelu SVM na danych testowych: 51.85%
target_ratio: {1: 1995, 2: 1995}
Dokładność mode